In [2]:
with open("data.txt", "r",encoding="utf-8") as output:
   text = output.read()
text = text.replace('™','')
vocab =sorted(list(set(text)))

print('Lenght of text is:',len(text))
print('Length of vocalbulary is:',len(vocab))

Lenght of text is: 2911830
Length of vocalbulary is: 106


In [3]:
s2i = {ch:i for i,ch in enumerate(vocab)}
# s2i.pop('™')

i2s = {i:ch for i,ch in enumerate(vocab)}
# i2s.pop(106)

# def encoder(string):
#     lis = []
#     for s in string:
#         lis.append(s2i[s])
    
#     return lis
encoder = lambda s:[s2i[x] for x in s]

# def decoder(lis):
#     s=''
#     for i in lis:
#       s+=i2s[i]
#     return s
decoder = lambda l: ''.join([i2s[x] for x in l])


"""Look into sentencepiece encoding used by google
    https://github.com/google/sentencepiece"""

"""Also look into the encoder used by openAi the tictoken
    https://github.com/openai/tiktoken"""

'Also look into the encoder used by openAi the tictoken\n    https://github.com/openai/tiktoken'

In [4]:
import torch

tensor = torch.tensor(encoder(text),dtype = torch.long)
print(tensor.dtype,tensor.shape)
print(tensor[:1000])

torch.int64 torch.Size([2911830])
tensor([30, 62, 64, 79, 60, 72, 68, 73, 74, 75, 67, 64, 73,  1, 41, 64, 81, 64,
        71,  1, 25,  1, 16, 52, 67, 60, 79,  1, 68, 78,  1, 60, 73,  1, 60, 62,
        64, 79, 60, 72, 68, 73, 74, 75, 67, 64, 73,  1, 71, 64, 81, 64, 71,  1,
        79, 64, 78, 79, 29,  1, 49, 67, 68, 78,  1, 79, 64, 78, 79,  1, 72, 64,
        60, 78, 80, 77, 64, 78,  1, 79, 67, 64,  1, 60, 72, 74, 80, 73, 79,  1,
        74, 65,  1, 60, 62, 64, 79, 60, 72, 68, 73, 74, 75, 67, 64, 73,  1, 68,
        73,  1, 79, 67, 64,  1, 61, 71, 74, 74, 63, 13,  1, 30, 62, 64, 79, 60,
        72, 68, 73, 74, 75, 67, 64, 73,  1, 68, 78,  1, 74, 73, 64,  1, 74, 65,
         1, 79, 67, 64,  1, 72, 74, 78, 79,  1, 62, 74, 72, 72, 74, 73,  1, 72,
        64, 63, 68, 62, 68, 73, 64, 78,  1, 80, 78, 64, 63,  1, 68, 73,  1, 74,
        81, 64, 77, 12, 79, 67, 64, 12, 62, 74, 80, 73, 79, 64, 77,  1, 75, 60,
        68, 73,  1, 77, 64, 71, 68, 64, 81, 64, 77, 78,  1, 60, 73, 63,  1, 65,
      

In [10]:
block_size = 12
batch_size = 6

ix = torch.randint(len(tensor)-block_size,[batch_size])
x = torch .stack([tensor[i:block_size+i] for i in ix])
y = torch .stack([tensor[i+1:block_size+i+1] for i in ix])

print(x.shape)
print(x)
print(y.shape)
print(y)

torch.Size([6, 12])
tensor([[73, 59, 17, 17, 11, 63, 68, 60, 61, 64, 79, 64],
        [79, 60, 67, 13, 64, 63, 80, 14, 73, 64, 80, 77],
        [ 1, 52, 60, 78, 67, 68, 73, 66, 79, 74, 73,  1],
        [63, 68, 78, 74, 77, 63, 64, 77, 13,  1, 48, 84],
        [45, 51,  1, 33, 43, 30,  1, 49, 64, 78, 79, 26],
        [ 1, 62, 60, 73,  1, 78, 67, 74, 82,  1, 68, 65]])
torch.Size([6, 12])
tensor([[59, 17, 17, 11, 63, 68, 60, 61, 64, 79, 64, 78],
        [60, 67, 13, 64, 63, 80, 14, 73, 64, 80, 77, 74],
        [52, 60, 78, 67, 68, 73, 66, 79, 74, 73,  1, 33],
        [68, 78, 74, 77, 63, 64, 77, 13,  1, 48, 84, 72],
        [51,  1, 33, 43, 30,  1, 49, 64, 78, 79, 26,  1],
        [62, 60, 73,  1, 78, 67, 74, 82,  1, 68, 65,  1]])


In [28]:
import token
import torch.nn as nn


class BinegramModel(nn.Module):
    
   def __init__(self,vocab_size):
      super().__init__()
      self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)

   def forward(self,idx,target=None):

      logits = self.token_embedding_table(idx)

      if target:
         log_loss = None
      else:
         batch,time,channel = logits.shape
         logits = logits.view(batch*time,channel)
         target = target.view(batch*time)      
         log_loss = nn.functional.cross_entropy(logits,target)
   
      return logits,log_loss
   

m = BinegramModel(len(vocab))
out,log = m(x,y)
print(log,out.shape)


tensor(5.2852, grad_fn=<NllLossBackward0>) torch.Size([72, 106])


In [32]:
nn.Embedding(len(vocab),len(vocab))(x)[0]

tensor([[ 1.1112, -1.6572, -1.6552,  ..., -0.7183,  0.4691, -0.5346],
        [-0.2798,  0.3794, -1.2251,  ..., -1.1613,  0.8229,  0.0327],
        [ 0.8097, -2.9785,  0.6784,  ..., -0.0153,  0.8220, -1.2269],
        ...,
        [-0.3654, -0.1509, -0.3544,  ..., -0.0411, -0.2526, -0.9263],
        [-0.5963,  0.0309,  1.6200,  ...,  1.2702, -1.5638, -0.3858],
        [-0.3654, -0.1509, -0.3544,  ..., -0.0411, -0.2526, -0.9263]],
       grad_fn=<SelectBackward0>)